In [1]:
import json

In [2]:
json_file = json.load(open('data_level0.json',encoding="utf-8"))

In [15]:
# Index 0: Test file (https://github.com/declare-lab/RECCON/blob/main/data/original_annotation/dailydialog_test.json)
# Index 1: Train file (https://github.com/declare-lab/RECCON/blob/main/data/original_annotation/dailydialog_train.json)
# Index 2: Valid file (https://github.com/declare-lab/RECCON/blob/main/data/original_annotation/dailydialog_valid.json)
dataset_div = ['Test', 'Train', 'Validation']
for index, label in enumerate(dataset_div):
    print(f"{label} size: {len(json_file[index].keys())}")

Test size: 225
Train size: 834
Validation size: 47


In [8]:
# Test water with train set
train = json_file[1]

for key in train.

834